In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline

In [35]:
#dados_teste = pd.read_csv('../data/processed/test_data.csv')

In [29]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))


def prep_text(text):
    try:
        tokens = word_tokenize(text)
        words = [word.lower() for word in tokens if word.isalpha()]
        words = [word for word in words if not word in stop_words]
    except:
        print(text)
        return None
    return ' '.join(words)

In [53]:
dados_treino = pd.read_csv('../data/processed/train_data.csv')
dados_treino.columns

Index(['name', 'category_1', 'category_2', 'category_3', 'item_condition_id',
       'brand_name', 'price', 'shipping', 'item_description', 'date', 'stock'],
      dtype='object')

In [71]:
dados_treino = pd.read_csv('../data/processed/train_data.csv')
dados_treino = dados_treino[['category_1', 'item_description', 'price']]
dados_treino.dropna(inplace=True)
dados_treino['item_description'] = dados_treino.apply(lambda x: prep_text(x['item_description']), axis=1)
#dados_teste = dados_teste[['name', 'price']]

In [149]:
modelos = {}

for category in dados_treino.category_1.unique():
    pipe = Pipeline([('tiidf', TfidfVectorizer(min_df = 2, max_df = 0.5, ngram_range = (1, 2))),
                    ('xgboost', XGBRegressor(random_state=101, n_jobs=-1))])
    modelos[category] = pipe
    X_treino = dados_treino.query(f'category_1 == "{category}"')['item_description']
    y_treino = dados_treino.query(f'category_1 == "{category}"')['price']
    # pipe.fit(X_treino, y_treino)
    modelos[category].fit(X_treino, y_treino)


In [104]:
dados_teste = pd.read_csv('../data/processed/test_data.csv')
dados_teste = dados_teste[['category_1', 'item_description', 'price']]
dados_teste.dropna(inplace=True)
dados_teste['item_description'] = dados_teste.apply(lambda x: prep_text(x['item_description']), axis=1)
#dados_teste = dados_teste.query('category_1 == "Electronics"')
# teste_features = tfidf.transform(dados_teste['item_description'])
dados_teste = dados_teste.assign(pred=0)

In [150]:
for category in dados_teste.category_1.unique():
    index = dados_teste.query(f'category_1 == "{category}"').index
    dados_teste.loc[index, 'pred'] = modelos[category].predict(dados_teste.loc[index]['item_description'])

In [151]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_squared_log_error

print('R² =', r2_score(dados_teste['price'], dados_teste['pred']))
print('MAE =', mean_absolute_error(dados_teste['price'], dados_teste['pred']))
print('RMSE =', mean_squared_error(dados_teste['price'], dados_teste['pred'])**0.5)

R² = 0.2930297131806632
MAE = 14.787244064681808
RMSE = 32.26328297148539


R² = 0.2873256500595014

MAE = 15.112203329937804

RMSE = 32.387622596149676

MSLE = 1048.9580974306252

In [152]:
for category in dados_teste.category_1.unique():
    tt = dados_teste.query(f'category_1 == "{category}"')
    print('='*30)
    print(category)
    print('R² =', r2_score(tt['price'], tt['pred']))
    print('MAE =', mean_absolute_error(tt['price'], tt['pred']))
    print('RMSE =', mean_squared_error(tt['price'], tt['pred'])**0.5)
    print('-'*30)

Women
R² = 0.26285147035854073
MAE = 15.73124475293124
RMSE = 33.6001476637003
------------------------------
Men
R² = 0.3427111380125816
MAE = 18.62455876240904
RMSE = 32.48888111939868
------------------------------
Vintage & Collectibles
R² = 0.14035060179620873
MAE = 18.391266892477844
RMSE = 47.02453678624211
------------------------------
Electronics
R² = 0.40522023335434554
MAE = 23.027521566483163
RMSE = 50.75747774768436
------------------------------
Beauty
R² = 0.09692333153455834
MAE = 9.82482874245746
RMSE = 23.032197644657565
------------------------------
Kids
R² = 0.18605937940758788
MAE = 10.798982945945314
RMSE = 20.0707243428561
------------------------------
Other
R² = 0.18689746916826
MAE = 12.141660281252504
RMSE = 24.762056832545852
------------------------------
Home
R² = 0.19085597707073898
MAE = 12.445126319198941
RMSE = 20.958082377047678
------------------------------
Sports & Outdoors
R² = 0.22921174385090992
MAE = 13.35321631667016
RMSE = 26.08098826777332